In [6]:
import numpy as np
import re
import csv

class Data(object):
  def __init__ (self, data_source,
                alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}",
                input_size=1014, num_of_classes=4):
    self.alphabet = alphabet
    self.alphabet_size = len(self.alphabet)
    self.dict = {}
    self.no_of_classes = num_of_classes
    for idx, char in enumerate(self.alphabet):
      self.dict[char] = idx + 1
    self.length = input_size
    self.data_source = data_source

  def load_data(self):
    data = []
    with open(self.data_source, 'r', encoding='utf-8') as f:
      rdr = csv.reader(f, delimiter=",", quotechar='"')
      for row in rdr:
        txt = ""
        for s in row[1:]:
          txt = txt + " " + re.sub("^\s*(.-)\s*$", "%1", s).replace("\\n", "\n")
        data.append((int(row[0]), txt))
      self.data = np.array(data)
      print("Data loaded from "+ self.data_source)
      print(len(data))
      print(data[:10])

  def get_all_data(self):
      data_size = len(self.data)
      start_index = 0
      end_index = data_size
      batch_texts = self.data[start_index:end_index]
      batch_indices = []
      one_hot = np.eye(self.no_of_classes, dtype='int64')
      classes = []
      for c, s in batch_texts:
        batch_indices.append(self.str_on_indexes(s))
        c = int(c) - 1
        classes.append(one_hot[c])
      return np.asarray(batch_indices, dtype='int64'), np.array(classes)

  def str_on_indexes(self, s):
      s = s.lower()
      max_length = min(len(s), self.length)
      str2idx = np.zeros(self.length, dtype='int64')
      for i in range(1, max_length + 1):
        c = s[-i]
        if c in self.dict:
          str2idx[i-1] = self.dict[c]
      return str2idx

In [7]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers import AlphaDropout
from keras.callbacks import TensorBoard

class CharCNNKim(object):
  def __init__ (self, input_size, alphabet_size, embedding_size,
                conv_layers, fully_connected_layers,
                num_of_classes, dropout_p,
                optimizer='adam', loss='categorical_crossentropy'):
    self.input_size = input_size
    self.alphabet_size = alphabet_size
    self.embedding_size = embedding_size
    self.conv_layers = conv_layers
    self.fully_connected_layers = fully_connected_layers
    self.num_of_classes = num_of_classes
    self.dropout_p = dropout_p
    self.optimizer = optimizer
    self.loss = loss
    self._build_model()

  def _build_model(self):
    inputs = Input(shape=(self.input_size,), name='sent_input', dtype='int64')
    x = Embedding(self.alphabet_size + 1, self.embedding_size, input_length=self.input_size)(inputs)
    
    convolution_output = []
    for num_filters, filter_width in self.conv_layers:
      conv = Conv1D(filters = num_filters,
                           kernel_size = filter_width,
                           activation='tanh',
                           name = 'Conv1D_{}_{}'.format(num_filters, filter_width))(x)
      pool = GlobalMaxPooling1D(name='MaxPoolingOverTime_{}_{}'.format(num_filters, filter_width))(conv)
      convolution_output.append(pool)
    x = Concatenate()(convolution_output)
    for f1 in self.fully_connected_layers:
      x = Dense(f1, activation='selu', kernel_initializer = 'lecun_normal')(x)
      x = AlphaDropout(self.dropout_p)(x)
    predictions = Dense(self.num_of_classes, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    self.model = model
    print("Built! ") 
    self.model.summary()

  def train(self, training_inputs, training_labels, validation_inputs, validation_labels, epochs, batch_size, checkpoint_every=100):
    logdir = '/tmp/logs'
    histogram_freq=checkpoint_every
    tensorboard = TensorBoard(batch_size, logdir ,histogram_freq,
                              write_graph=False, write_grads=True, write_images=False,
                              embeddings_freq=checkpoint_every,
                              embeddings_layer_names=None)
    print("Training CharCNNKim Model: ")
    self.model.fit(training_inputs, training_labels,
                   validation_data=(validation_inputs, validation_labels),
                   epochs=epochs,
                   batch_size=batch_size,
                   verbose=2)
    
  def test(self, testing_inputs, testing_labels, batch_size):
    self.model.evaluate(testing_inputs, testing_labels, batch_size=batch_size, verbose=1)
    self.model.predict(testing_inputs, batch_size=batch_size, verbose=1)



In [8]:
import tensorflow as tf
import json

if __name__ == "__main__":
  config = json.load(open("config.json"))
  training_data = Data(data_source=config["data"]["training_data_source"],
                       alphabet=config["data"]["alphabet"],
                       input_size=config["data"]["input_size"],
                       num_of_classes=config["data"]["num_of_classes"])
  training_data.load_data()
  training_inputs, training_labels = training_data.get_all_data()
  validation_data = Data(data_source=config["data"]["validation_data_source"],
                         alphabet=config["data"]["alphabet"],
                         input_size=config["data"]["input_size"],
                         num_of_classes=config["data"]["num_of_classes"])
  validation_data.load_data()
  validation_inputs, validation_labels = validation_data.get_all_data()

  model = CharCNNKim(input_size=config["data"]["input_size"],
                     alphabet_size=config["data"]["alphabet_size"],
                     embedding_size=config["char_cnn_kim"]["embedding_size"],
                     conv_layers=config["char_cnn_kim"]["conv_layers"],
                     fully_connected_layers=config["char_cnn_kim"]["fully_connected_layers"],
                     num_of_classes=config["data"]["num_of_classes"],
                     dropout_p=config["char_cnn_kim"]["dropout_p"],
                     optimizer=config["char_cnn_kim"]["optimizer"],
                     loss=config["char_cnn_kim"]["loss"])
  
  model.train(training_inputs = training_inputs,
              training_labels = training_labels,
              validation_inputs = validation_inputs,
              validation_labels = validation_labels,
              epochs = config["training"]["epochs"],
              batch_size = config["training"]["batch_size"],
              checkpoint_every = config["training"]["checkpoint_every"])

Data loaded from C:\Users\Sagar Mishra\Desktop\Data\CLE\train.csv
120000
[(3, " Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."), (3, ' Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'), (3, " Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums."), (3, ' Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official

InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.